#### Import Libraries and Load Data

In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#### Load the Data

In [ ]:
# Load the training and test datasets
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

#### Show data

In [ ]:
print('First 5 rows of training data')
train_df.head(2)

#### Basic Dataset Information

In [ ]:
# Get basic information about the dataset
print("Dataset Info:")
print(f"Number of features: {len(train_df.columns)}")
print(f"Number of training samples: {len(train_df)}")
print(f"Number of test samples: {len(test_df)}")

print("\nColumn names:")
print(train_df.columns.tolist())